# Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

In [2]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [3]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:

                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                feature_indices = np.random.permutation(np.arange(D))[:D_try]
                left, right = make_density_split_node(node, N, feature_indices)
                stack += [left, right]
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)
        
        return self

    def predict(self, x):
        m, M = self.box
        
        if np.any(x > M) or np.any(x < m):
            return 0

        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        return leaf.response * self.prior

<div style="color: green; font-weight: bold">Did not limit the random permuation of features to the previously computed set of valid features</div>
<div style="color: green; font-weight: bold">Prediction is very similar to sample solution</div>

In [4]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    
    for feat in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        # It is necessary because otherwise we would recieve a threshold candidates
        # on a feature value not between two of them
        data_unique = np.sort(np.unique(node.data[:, feat]), axis=0)
        # Compute candidate thresholds
        tj = (data_unique[1:] + data_unique[:-1]) / 2
        
        for t in tj:
            loo_error = ( leave_one_out_error(node.data, node.data[node.data[:, feat] <= t], t, N)
                        + leave_one_out_error(node.data, node.data[node.data[:, feat] > t], t, N))

            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                node.feature = feat
                node.threshold = t

    # create children
    node.left = Node()
    node.right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    is_left = node.data[:, node.feature] < node.threshold
    node.left.data = node.data[is_left]
    node.left.box = (node.left.data.min(axis=0), node.left.data.max(axis=0))
    node.right.data = node.data[~is_left]
    node.right.box = (node.right.data.min(axis=0), node.right.data.max(axis=0))
    
    return node.left, node.right


def leave_one_out_error(data, data_below_threshold, t, N):
    N_l = data_below_threshold.shape[0]
    
    M_l = data_below_threshold.max(axis=0)
    m_l = data_below_threshold.min(axis=0)
    
    # sadly this also works to fix the box of the node
    M_l += M_l == m_l

    V_l = np.product(M_l - m_l)
    return (N_l / (N * V_l)) * ((N_l / N) - 2 * (N_l - 1) / (N - 1))


<div style="color: green; font-weight: bold">Computation of candidate thresholds very similar to sample solution</div>
<div style="color: green; font-weight: bold">Leave one out error implemented in helper function, small differences in computation of required variables</div>
<div style="color: green; font-weight: bold">Initialization of left and right nodes also similar to sample solution</div>

In [5]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    node.response = n / N

<div style="color: green; font-weight: bold">Did not divide by volume</div>

# Decision Tree

In [6]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                feature_indices = np.random.permutation(np.arange(D))[:D_try]
                left, right = make_decision_split_node(node, feature_indices)
                stack += [left, right]
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node)
        
        return self
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response

<div style="color: green; font-weight: bold">Very similar to sample solution</div>

In [7]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    node.feature = None
    node.threshold = None
    
    for feat in feature_indices:
        data_unique = np.sort(np.unique(node.data[:, feat]), axis=0)

        # Compute candidate thresholds
        tj = (data_unique[1:] + data_unique[:-1]) / 2
        
        for t in tj:
            gi_error = (gini_impurity(node.labels[node.data[:, feat] <= t])
                      + gini_impurity(node.labels[node.data[:, feat] > t]))
            
            if gi_error < e_min:
                e_min = gi_error
                node.feature = feat
                node.threshold = t
    
    # create children
    node.left = Node()
    node.right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above    
    is_left = node.data[:, node.feature] < node.threshold
    node.left.data = node.data[is_left]
    node.left.labels = node.labels[is_left]
    node.right.data = node.data[~is_left]
    node.right.labels = node.labels[~is_left]
    
    # return the children (to be placed on the stack)
    return node.left, node.right

def gini_impurity(labels):
    N_l = labels.shape[0]
    return N_l * (1 - np.sum(np.square((np.unique(labels)[:, None] == labels).sum(axis=1)) / (N_l ** 2)))

<div style="color: green; font-weight: bold">Also very similar to sample solution</div>

In [8]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = node.data.shape[0]
    most_frequent_label = np.argmax(np.bincount(node.labels))
    node.response = most_frequent_label

<div style="color: green; font-weight: bold">We return a single label, whereas the sample solution returns a probability distribution</div>

In [9]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return len(np.unique(node.labels)) == 1

# Evaluation of Density and Decision Tree

In [10]:
# read and prepare the digits data
digits = load_digits()
data = digits.data
images = digits.images
target = digits.target
target_names = digits.target_names

In [11]:
# train trees, plot training error confusion matrices, and comment on your results


In [12]:
density_trees = [DensityTree().train(data[target == label], len(data[target == label]) / len(data), 10) 
                     for label in np.unique(target)]

In [13]:
density_predictions = np.array([[tree.predict(row) for tree in density_trees] for row in data]).argmax(axis=1)

In [14]:
print(f"training error rate: {100 * sum(density_predictions != target) / len(target):.2f}%")
print("compared to 90% error for pure guessing this is not too bad :)")

for label in range(10):
    print("\n")
    print(f"\t Predicted {label} | Predicted not {label}")
    print("\t -------------------------------")
    print(f"Is {label}     |  {100 * sum(density_predictions[target == label] == label) / len(target):.2f}%   |  {100 * sum(density_predictions[target == label] != label)/ len(target):.2f}             ")
    print(f"Is not {label} |  {100 * sum(density_predictions[target != label] == label)/ len(target):.2f}%   |  {100 * sum(density_predictions[target != label] != label)/ len(target):.2f}             ")


training error rate: 40.62%
compared to 90% error for pure guessing this is not too bad :)


	 Predicted 0 | Predicted not 0
	 -------------------------------
Is 0     |  6.96%   |  2.95             
Is not 0 |  0.00%   |  90.09             


	 Predicted 1 | Predicted not 1
	 -------------------------------
Is 1     |  5.56%   |  4.56             
Is not 1 |  7.51%   |  82.36             


	 Predicted 2 | Predicted not 2
	 -------------------------------
Is 2     |  6.57%   |  3.28             
Is not 2 |  5.34%   |  84.81             


	 Predicted 3 | Predicted not 3
	 -------------------------------
Is 3     |  5.68%   |  4.51             
Is not 3 |  3.06%   |  86.76             


	 Predicted 4 | Predicted not 4
	 -------------------------------
Is 4     |  7.57%   |  2.50             
Is not 4 |  5.45%   |  84.47             


	 Predicted 5 | Predicted not 5
	 -------------------------------
Is 5     |  7.85%   |  2.28             
Is not 5 |  10.29%   |  79.58             




In [15]:
decision_tree = DecisionTree().train(data, target)
decision_predictions = np.array([decision_tree.predict(row) for row in data])

for label in range(10):
    print("\n")
    print(f"\t Predicted {label} | Predicted not {label}")
    print("\t -------------------------------")
    print(f"Is {label}     |  {100 * sum(decision_predictions[target == label] == label) / len(target):.2f}%   |  {100 * sum(decision_predictions[target == label] != label)/ len(target):.2f}             ")
    print(f"Is not {label} |  {100 * sum(decision_predictions[target != label] == label)/ len(target):.2f}%   |  {100 * sum(decision_predictions[target != label] != label)/ len(target):.2f}             ")




	 Predicted 0 | Predicted not 0
	 -------------------------------
Is 0     |  9.46%   |  0.45             
Is not 0 |  1.17%   |  88.93             


	 Predicted 1 | Predicted not 1
	 -------------------------------
Is 1     |  9.29%   |  0.83             
Is not 1 |  2.23%   |  87.65             


	 Predicted 2 | Predicted not 2
	 -------------------------------
Is 2     |  8.35%   |  1.50             
Is not 2 |  0.72%   |  89.43             


	 Predicted 3 | Predicted not 3
	 -------------------------------
Is 3     |  8.57%   |  1.61             
Is not 3 |  1.00%   |  88.81             


	 Predicted 4 | Predicted not 4
	 -------------------------------
Is 4     |  8.68%   |  1.39             
Is not 4 |  0.67%   |  89.26             


	 Predicted 5 | Predicted not 5
	 -------------------------------
Is 5     |  8.85%   |  1.28             
Is not 5 |  1.28%   |  88.59             


	 Predicted 6 | Predicted not 6
	 -------------------------------
Is 6     |  9.13%   |  0.9

In [16]:
print(f"training error rate: {100 * sum(decision_predictions != target) / len(target):.2f}%")
print("this is a lot better than the density predictions. It probably helps if an algorithm \"knows\" about the training labels")

training error rate: 12.41%
this is a lot better than the density predictions. It probably helps if an algorithm "knows" about the training labels


# Density and Decision Forest

In [17]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            data_selection_indices = np.random.choice(data.shape[0], size=data.shape[0], replace=True)
            tree.train(data[data_selection_indices], prior, n_min)

    def predict(self, x):
        # compute the ensemble prediction
        return sum([tree.predict(x) for tree in self.trees]) / len(self.trees) 

<div style="color: green; font-weight: bold">Added replace=True attribute, but it is true by default.</div>
<div style="color: green; font-weight: bold">The mean was computed manually.</div>
<div style="color: green; font-weight: bold">Otherwise very similar to sample solution.</div>

In [18]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            data_selection_indices = np.random.choice(data.shape[0], size=data.shape[0], replace=True)
            tree.train(data[data_selection_indices], labels[data_selection_indices], n_min)
    def predict(self, x):
        # compute the ensemble prediction
        predictions = np.array([tree.predict(x) for tree in self.trees])
        return np.argmax(np.bincount(predictions))[0]

<div style="color: green; font-weight: bold">We return an individual label as prediction, whereas the sample solution returns a probability distribution.</div>
<div style="color: green; font-weight: bold">Otherwise very similar to sample solution</div>

# Evaluation of Density and Decision Forest

In [19]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
digits = load_digits()

densityForests = []
for i in range(10):
    data = digits.data[digits.target == i]
    prior = data.shape[0] / digits.data.shape[0]
    forest = DensityForest(20)
    forest.train(data, prior)
    densityForests.append(forest)

KeyboardInterrupt: 

In [ ]:
def predictDensityForest(x):
    max_response = float("inf")
    label = None
    for (index, forest) in enumerate(densityForests):
        response = forest.predict(x)
        print(response)
        if response > max_response:
            max_response = response
            label = index
    return label

print(predictDensityForest(digits.data[2]))

<div style="color: green; font-weight: bold">We did not finish the evaluation task.</div>